# Libraries and Dependencies

In [1]:
import os
import random
from concurrent.futures import ThreadPoolExecutor

import cv2 as cv
import numpy as np
import pyvips
import matplotlib.pyplot as plt
import plotly.express as px
from tqdm import tqdm

# Utilities and Helper functions

In [2]:
def highest_divisor(n):
    if n <= 1:
        return None  
    
    for i in range(int(n**0.5), 0, -1):
        if n % i == 0:
            return n // i if n // i != n else i

In [3]:
def apply_gamma_correction(img, gamma):
  lookUpTable = np.empty((1,256), np.uint8)
  for i in range(256):
      lookUpTable[0,i] = np.clip(pow(i / 255.0, gamma) * 255.0, 0, 255)
  return cv.LUT(img, lookUpTable)

In [4]:
def pyvips_to_numpy(vips_image):
    return np.ndarray(buffer=vips_image.write_to_memory(),
                      dtype=np.uint8,
                      shape=[vips_image.height, vips_image.width, vips_image.bands])

In [5]:
def process_tile(slide, width, height, x, y, patch_size_w, patch_size_h, lower_bnd_intensity, upper_bnd_intensity, fdir):
    patch_coordinates = []
    actual_patch_w = min(patch_size_w, width - x)
    actual_patch_h = min(patch_size_h, height - y)
    
    patch_coordinates.append((x, y, actual_patch_w, actual_patch_h))
    
    tile = slide.crop(x, y, actual_patch_w, actual_patch_h)

    tile_array = pyvips_to_numpy(tile)

    mean_value = np.mean(tile_array)

    if lower_bnd_intensity < mean_value <= upper_bnd_intensity:
        return None
    
    output_filename = f"{fdir}/tile_{x}_{y}.jpg"
    coordinates = x, y, x + patch_size_w, y + patch_size_h
    tile.write_to_file(output_filename)
    
    return output_filename

## The following tile generator function gets some inputs:
-   Image file path
-   Patch type (square, rectangle, or fixed):
    -   if fixed, fixed patch width:
    -   if fixed, fixed patch height:
-   Number of random tiles to generate:
-   Lower intensity bound
-   Upper intensity bound

In [68]:
def generate_random_tiles():
    image_path = os.path.expanduser(input("Enter the image file path: "))
    slide = pyvips.Image.new_from_file(image_path)
    width = slide.width
    height = slide.height
    
    patch_type = input("Enter patch type (square, rectangle, or fixed): ")

    fixed_w, fixed_h = None, None
    min_size, max_size = None, None

    if patch_type == "fixed":
        fixed_w = int(input("Enter fixed patch width: "))
        fixed_h = int(input("Enter fixed patch height: "))
        fdir = "Tile" + str(fixed_h) + str(fixed_w) + "_" + image_path[-6:-5]
    else:
        min_size = int(input("Enter the lower bound for patch size: "))
        max_size = int(input("Enter the upper bound for patch size: "))
        fdir = "Tile_" + str(min_size) +  "_" + str(max_size) + "_" + image_path[-6:-5]

    os.mkdir(fdir)
    num_tiles = int(input("Enter the number of random tiles to generate: "))
    lower_bnd_intensity = int(input("Enter the lower intensity bound: "))
    upper_bnd_intensity = int(input("Enter the upper intensity bound: "))

    # Use ThreadPoolExecutor for parallel processing
    with ThreadPoolExecutor() as executor:
        futures = []
        
        for _ in range(num_tiles):
            x = random.randint(0, width - 800) 
            y = random.randint(0, height - 800)
            
            if patch_type == "square":
                patch_size_w = patch_size_h = random.randint(min_size, max_size)
                
            elif patch_type == "rectangle":
                patch_size_w = random.randint(min_size, max_size)
                patch_size_h = random.randint(min_size, max_size)
                
            elif patch_type == "fixed":
                patch_size_w = fixed_w
                patch_size_h = fixed_h
                
            else:
                raise ValueError(f"Unknown patch type: {patch_type}")
            
            futures.append(executor.submit(process_tile, slide, width, height, x, y, patch_size_w, patch_size_h, lower_bnd_intensity, upper_bnd_intensity, fdir))

        # Wait for all tasks to complete
        for future in futures:
            result = future.result()
            if result:
                print(f"Saved: {result}")

In [69]:
generate_random_tiles()
# Enter the image file path:  ../../Data/ALI surgical/ALI surgical w catheter m #1.mrxs
# Enter patch type (square, rectangle, or fixed):  fixed
# Enter fixed patch width:  3000
# Enter fixed patch height:  1000
# Enter the number of random tiles to generate:  200
# Enter the lower intensity bound:  248
# Enter the upper intensity bound:  255

Saved: Tile_150_900_1/tile_36563_141653.jpg
Saved: Tile_150_900_1/tile_25265_82575.jpg
Saved: Tile_150_900_1/tile_50547_166788.jpg
Saved: Tile_150_900_1/tile_28682_56372.jpg
Saved: Tile_150_900_1/tile_42992_139381.jpg
Saved: Tile_150_900_1/tile_39844_167987.jpg
Saved: Tile_150_900_1/tile_48274_144619.jpg


# This version generates fixed tiles from slides by sliding the tile window throughout the entire slide.





-   Parallel execution of the code using 'ThreadPoolExecutor' and using maximum core

In [11]:
def process_tile_2(x, y, patch_size_w, patch_size_h, lower_bnd_intensity, upper_bnd_intensity, file_path, output_dir):
    patch_coordinates = []
    # Calculate the actual patch size (to handle edges)
    actual_patch_w = min(patch_size_w, width - x)
    actual_patch_h = min(patch_size_h, height - y)
    
    patch_coordinates.append((x, y, actual_patch_w, actual_patch_h))
    
    # Crop the tile
    tile = slide.crop(x, y, actual_patch_w, actual_patch_h)

    # Convert tile to NumPy array
    tile_array = pyvips_to_numpy(tile)

    # Calculate the mean using NumPy
    mean_value = np.mean(tile_array)

    # Skip if the mean is within the range to be ignored
    if lower_bnd_intensity < mean_value <= upper_bnd_intensity:
        return None
    
    # Write the tile to a file only if necessary
    output_filename = os.path.expanduser(f"{output_dir}/tile_{x}_{y}.png")
    coordinates = x, y, x + patch_size_w, y + patch_size_h
    tile.pngsave(output_filename, compression=9)
    
    return output_filename



In [12]:
def generate_tiles(patch_size_w, patch_size_h, lower_bnd_intensity, upper_bnd_intensity, file_path,output_dir):
    # Use ThreadPoolExecutor for parallel processing
    with ThreadPoolExecutor() as executor:
        futures = []
        for y in range(0, height, patch_size_h):
            for x in range(0, width, patch_size_w):
                futures.append(executor.submit(process_tile_2, x, y, patch_size_w, patch_size_h, lower_bnd_intensity, upper_bnd_intensity, file_path,output_dir))

        # Wait for all tasks to complete
        for future in futures:
            result = future.result()

In [13]:
patch_size_w = 1637
patch_size_h = 1018

base_slide_path = os.path.expanduser(f"~/Documents/Data/ALI surgical/Injured Slides/")
files = [file for file in os.listdir(base_slide_path) if file.endswith(".mrxs")]

for file in tqdm(files, desc="Generating Tiles .mrxs files"):
    full_path = os.path.join(base_slide_path, file)
    output_dir = os.path.expanduser(f"~/Documents/Code/Lung_Injury/Tiles_{patch_size_h}_{patch_size_w}_{file}")
    os.makedirs(output_dir, exist_ok=True) 
    slide = pyvips.Image.new_from_file(full_path)
    width = slide.width
    height = slide.height

    generate_tiles(
        patch_size_w=patch_size_w,
        patch_size_h=patch_size_h,
        lower_bnd_intensity=240,
        upper_bnd_intensity=255,
        file_path=full_path,
        output_dir=output_dir
    )

Generating Tiles .mrxs files:   0%|          | 0/10 [00:26<?, ?it/s]


KeyboardInterrupt: 